# Ceneo Scraper 

## Components of single opinion


|Component|Selector|Variable|
|---------|--------|--------|
|opinion ID|["data-entry-id"]|opinion_id|
|opinion’s author|span.user-post__author-name|author|
|author’s recommendation|span.user-post__author-recomendation > em|recommendation|
|score expressed in number of stars|span.user-post__score-count|score|
|opinion’s content|div.user-post__text|content|
|list of product advantages|div.review-feature__title review-feature__title--positives ~ div.review-feature__item|pros|
|list of product disadvantages|div.review-feature__title review-feature__title--negatives ~ div.review-feature__item|cons|
|how many users think that opinion was helpful|button.vote-yes > span|helpful|
|how many users think that opinion was unhelpful|button.vote-no > span|unhelpful|
|publishing date|span.user-post__published > time:nth-child(1).["datetime"]|post_date|
|purchase date|span.user-post__published > time:nth-child(2).["datetime"]|purchase_date|

# Imports

In [77]:
import requests
from bs4 import BeautifulSoup

In [80]:
def extract_content(ancestor, selector=None, attribute=None ,return_list=False):
    if selector:
        if return_list:
            if attribute:
                return [tag[attribute].strip() for tag in ancestor.select(selector)]
            return [tag.text.strip() for tag in ancestor.select(selector)]
        if attribute:
            try:
                return ancestor.select_one(selector)[attribute].strip()
            except TypeError:
                return None
        return ancestor.select_one(selector).text.strip()
    if attribute:
        return ancestor[attribute]
    return ancestor.text.strip()

## Opinion structure






In [83]:
print(single_opinion)

{'opinion_id': '17349010', 'author': 'KamilZ1', 'recommendation': 'Polecam', 'score': '5/5', 'content': 'W obudowie przypominającej niewielką drukarkę, kryje się urządzenie wielofunkcyjne, które posiada funkcję drukowania, skanowania oraz kopiowania. I właśnie niewielkie rozmiary i kompaktowość są największymi zaletami HP DeskJet 3762. Urządzenie nie zajmuje wiele miejsca i przez to idealnie nadaje się do domu, gdzie nie mamy wydzielonego pomieszczenia biurowego, a jedynie kącik do pracy. Kolejną zaletą jest na pewno cena. W tym przedziale cenowym trudno znaleźć urządzenie o takich parametrach i takiej wydajności. Jakość wydruków również pozytywnie zaskakuje, choć trzeba uczciwie przyznać, że nie jest to demon prędkości. Na plus można zaliczyć także drukowanie bezprzewodowe, które samo w sobie jest bardzo praktycznym rozwiązaniem, a w  HP DeskJet 3762 działa bez zarzutu. Jakość wykonania w porządku, design to rzecz gustu, ale fajnie jakby była możliwość wyboru kolorystyki – turkusowy n

In [81]:
selectors={



"opinion_id": ( None, "data-entry-id",),
"author": ( "span.user-post__author-name",),
"recommendation": ( "span.user-post__author-recomendation > em",),
"score":( "span.user-post__score-count",),
"content":( "div.user-post__text",),
"pros":("div.review-feature__title--positives ~ div.review-feature__item", None, True),
"cons":( "div.review-feature__title--negatives ~ div.review-feature__item",None,True),
"helpful":( "button.vote-yes > span",),
"unhelpful":( "button.vote-no > span",),
"post_date":("span.user-post__published > time:nth-child(1)","datetime"),
"purchase_date":( "span.user-post__published > time:nth-child(2)","datetime"),
}

# Send request to Ceneo.pl service

In [78]:
product_id = "108290707"
url = f"https://www.ceneo.pl/{product_id}#tab=reviews"
response = requests.get(url)
response.status_code

200

## Convert plain text HTML code into DOM structure

In [79]:
page_dom = BeautifulSoup(response.text, "html.parser")
opinions = page_dom.select("div.js_product-review")
opinion = page_dom.select_one("div.js_product-review")

In [82]:
single_opinion = {
    key:extract_content(opinion, *value)
        for key, value in selectors.items()
}